In [1]:
from dataclasses import fields

from build.lib.phenopacket_mapper.data_standards import DataField
from phenopacket_mapper.data_standards import DataModel, ValueSet, DataSection, OrGroup

In [ ]:
genomic_interpretation = DataModel(
    data_model_name="Phenopacket schema Genomic Interpretation",
    fields=(
        DataField(
            name="subject_or_biosample_id",
            specification=str,
            required=True,
            description="The id of the patient or biosample that is the subject being interpreted. REQUIRED."
        ),
        
        DataField(
            name="interpretation_status",
            specification=ValueSet(
                name="Interpretation Status Value Set",
                elements=["UNKNOWN_STATUS", "REJECTED", "CANDIDATE", "CONTRIBUTORY", "CAUSATIVE"],
            ),
            required=True,
            description="status of the interpretation. REQUIRED.",
        ),
        
        DataSection(
            name="call",
            fields=(
                OrGroup(
                    fields=(
                        DataSection(
                            name="GeneDescriptor",
                            fields=(
                                DataField(
                                    name="value_id",
                                    specification=str,
                                    required=True,
                                    description="Official identifier of the gene. REQUIRED."
                                ),

                                DataField(
                                    name="symbol",
                                    specification=str,
                                    required=True,
                                    description="Official gene symbol. REQUIRED."
                                ),

                                DataField(
                                    name="description",
                                    specification=str,
                                    required=False,
                                    description="A free-text description of the gene"
                                ),
                            ),
                        ),
                        
                        DataSection(
                            
                        ),
                    )
                ),
            ),
        ),
    )
)